In [9]:
from langchain_openai import ChatOpenAI
from typing import List
from typing import TypedDict, Annotated, Sequence
import os, operator, json
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents.agent_types import AgentType
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
import time
from dateutil import parser
from datetime import datetime


OPENAI_API_KEY="sk-vppgz5x"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [10]:
contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "05/31/2025",
  "Contract_amount": "$90,000.00"}"""

In [11]:
llm = ChatOpenAI(model="gpt-4-1106-preview")

# Create Agent Using Customized Function

In [160]:
def month_number_to_date(month_number, start_year=2023, start_month=5):
    # Adjust month_number for the start_month
    adjusted_month_number = month_number + start_month -1
    
    # Calculate the actual year and month considering the start_month
    year = start_year + (adjusted_month_number - 1) // 12
    month = (adjusted_month_number - 1) % 12 + 1
    
    return datetime(year, month, 1)

In [161]:
def parse_date(date_str, dayfirst=False):
    return parser.parse(date_str, dayfirst=dayfirst)

In [162]:
def getting_yearly_amount(contract):
#def getting_yearly_amount(contract=f"{contract}"):
    try:
        contract = json.loads(contract)
    except json.JSONDecodeError as e:
        print('Failed to decode coefficients:', e)
        return None
    # Extracting required information from the contract
    if len(contract['Term_start_date']) <= 5:
        start_month = int(contract['Term_start_date'])
        end_month = int(contract['Term_end_date'])
        start_year = int(contract['Contract_Year'])
        term_start_date=datetime(start_year, start_month, 1)
        term_end_date = month_number_to_date(month_number=end_month, start_year=start_year, start_month=start_month)
        #print(term_start_date, term_end_date)
    else:
        term_start_date = parse_date(contract['Term_start_date'])
        term_end_date = parse_date(contract['Term_end_date'])

    if type(contract['Contract_amount']) == list:
        contract_amount = sum([float(item['Amount'].replace('$', '').replace(',', '')) for item in contract['Contract_amount']])
    else: 
        contract_amount = float(contract['Contract_amount'].replace('$', '').replace(',', ''))

    # Calculating the total number of months for the contract period
    total_months = (term_end_date.year - term_start_date.year) * 12 + (term_end_date.month - term_start_date.month + 1)

    print(total_months)

    # Finding the total number of years
    total_years = term_end_date.year - term_start_date.year + 1

    # Getting the number of months in each year
    months_per_year = {}
    for year in range(term_start_date.year, term_end_date.year + 1):
        if year == term_start_date.year:
            months_per_year[year] = 12 - term_start_date.month + 1
        elif year == term_end_date.year:
            months_per_year[year] = term_end_date.month
        else:
            months_per_year[year] = 12

    # Calculating the amount corresponds to each month
    amount_per_month = contract_amount / total_months

    # print(amount_per_month)
    # print(months_per_year)

    # Calculating the amount corresponds to each year
    amount_per_year = {year: round(months * amount_per_month,2) for year, months in months_per_year.items()}

    return amount_per_year

equation_tool = Tool(name='Yearly Amount Estimation', func=getting_yearly_amount, description="Useful for data provided in the json format and estimating amount assciated for each year.")

tools = [equation_tool]

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3, input_key="input", output_key="output",return_messages=True) 

agent_parameters = {
    'agent':'chat-conversational-react-description',
    #'agent':AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    'tools':tools,
    'llm':llm,
    'verbose':True,
    'max_iterations':4,
    'return_intermediate_steps':True,
    'early_stopping_method':'generate',
  
    'memory':memory
}

agent = initialize_agent(**agent_parameters)


In [163]:
contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "05/31/2025",
  "Contract_amount": "$90,000.00"}"""

contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Year": "2023",
  "Contract_Date": "5",
  "Term_start_date": "12",
  "Term_end_date": "36",
  "Contract_amount": "$90,000.00"}"""

contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Year": "2023",
  "Contract_Date": "5",
  "Term_start_date": "12",
  "Term_end_date": "36",
  "Contract_amount":[{"item 1" : "SD Card", "Quantity" : "5", "Amount" : "30$"}, {"item 2": "Tea Bag", "Quantity": "10", "Amount": "5$"} ]
  }"""



In [164]:
%time
output= agent.invoke({"input":f"get each year amount from the given contract {contract} "})


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


> Entering new AgentExecutor chain...
```json
{
    "action": "Yearly Amount Estimation",
    "action_input": "{\"Sno\": 1, \"Contract_Number\": \"C887/36\", \"Contract_Year\": \"2023\", \"Contract_Date\": \"5\", \"Term_start_date\": \"12\", \"Term_end_date\": \"36\", \"Contract_amount\":[{\"item 1\" : \"SD Card\", \"Quantity\" : \"5\", \"Amount\" : \"30$\"}, {\"item 2\": \"Tea Bag\", \"Quantity\": \"10\", \"Amount\": \"5$\"}]}"
}
```36

Observation: {2023: 0.97, 2024: 11.67, 2025: 11.67, 2026: 10.69}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The estimated contract amounts per year are as follows: $0.97 for 2023, $11.67 for 2024, $11.67 for 2025, and $10.69 for 2026."
}
```

> Finished chain.


In [165]:
result=output['intermediate_steps'][-1][-1]
result

{2023: 0.97, 2024: 11.67, 2025: 11.67, 2026: 10.69}